# Exemplo de programação de operações em Job Shop

Programa que lê um conjunto de tempos de processamento de tarefas em um ambiente Job Shop e uma sequencia. Ele deve retornar o makespan.

Importação das bibliotecas 

In [25]:
import numpy as np
import plotly.io as pio
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import plotly.graph_objs as go# for Gantt
from plotly.offline import download_plotlyjs, init_notebook_mode,  plot #for Gantt
from collections import defaultdict
import random
import datetime                         #for Gantt (time formatting) #duration is in days
import plotly                           #for Gantt (full package)
import plotly.figure_factory as ff      #for Gantt
import chart_studio.plotly as py  
from itertools import product
from mip import Model, BINARY

Leitura dos dados

In [26]:
# Leitura da matriz dos tempos de processamento

tempos_Proc = []
fob = open("tempos_processamento-Job_shop.txt", "r") #Lendo matriz. Desse jeito, tempos_Proc[n][m]
for linha in fob.readlines():
    #print("A linha é: ",linha)
    tempos_Proc.append([int(i) for i in linha.split()])
fob.close()

tempos_Proc = np.array(tempos_Proc)
print(tempos_Proc)

[[ 5  7 10]
 [ 9  5  3]
 [ 5  8  2]
 [ 2  7  4]
 [ 8  8  8]]


In [27]:
# Leitura da matriz de roteiro de fabricação

roteiro_Fabr = []
fob = open("matriz_roteiro-Job_shop.txt", "r") #Lendo matriz. Desse jeito, tempos_Proc[n][m]
for linha in fob.readlines():
    #print("A linha é: ",linha)
    roteiro_Fabr.append([int(i) for i in linha.split()])
fob.close()

roteiro_Fabr = np.array(roteiro_Fabr)
print(roteiro_Fabr)

[[1 0 2]
 [0 1 2]
 [2 1 0]
 [1 0 2]
 [2 0 1]]


Declaração dos conjuntos

In [28]:
job =  len(tempos_Proc[0]) # número de jobs
machine =  len(tempos_Proc) # número de máquinas

print(f' O número de jobs a serem sequenciados é {job} e o número de máquinas é {machine}')

# Conjunto de tarefas
n = len(tempos_Proc) # número de jobs
print(n)

# Conjunto de máquinas
m =  len(tempos_Proc[0]) # número de máquinas
print(m)


 O número de jobs a serem sequenciados é 3 e o número de máquinas é 5
5
3


Declaração dos parâmetros

In [29]:
# Parâmetro big M - Utilizado nas restrições disjuntivas para sequenciamento

M = np.max(np.cumsum(tempos_Proc))

print(M)

91


Criação do modelo de otimização

In [30]:
model = Model('JSSP')

Declaração das variáveis

In [31]:
# Variável de decisão: C - instante de térmo da tarefa i na última máquina k da sequência de programação
C = model.add_var(name="C")

# Variável de decisão binária: Y_ijk = 1 se a tarefa i precede a tarefa j na máquina k
Y = [[[model.add_var(var_type=BINARY, name='Y({},{},{})'.format(j+1, k+1, i+1))
       for i in range(m)] for k in range(n)] for j in range(n)]

# Variável de decisão X_ij - instante de início da tarefa i na máquina j
X = [[model.add_var(name='X({},{})'.format(j+1, i+1))
      for i in range(m)] for j in range(n)]


Declaração da Função Objetivo

$min C$

$C \ge X_{j,o_{m,j}} + p_{j,o_{m,j}}, \qquad \forall j \in n$

Onde:

$o_{m,j}$ representa a operação respectiva ao roteriro do job $j$ na máquina $m$ - estas informações estão no parâmetro roteiro_Fabr

$p_{m,j}$ representa o tempo de processamento do job $j$ na máquina $m$ - estas informações estão no parâmetro tempos_Proc

In [32]:
model.objective = C

for j in range(n):
    model += C - X[j][roteiro_Fabr[j][m - 1]] >= tempos_Proc[j][roteiro_Fabr[j][m - 1]]

Declaração das restrições

$X_{j,o_{r,j}} \ge X_{j,o_{r-1,j}} + p_{j,o_{r-1,j}}, \qquad \forall j \in n, \forall r \in \{2,3,...m\}$


In [33]:
for (j, i) in product(range(n), range(1, m)):
    model += X[j][roteiro_Fabr[j][i]] - X[j][roteiro_Fabr[j][i-1]] >= \
        tempos_Proc[j][roteiro_Fabr[j][i-1]]

$X_{i,j} \ge X_{i,k} + p_{i,k} - M(Y_{i,j,k}), \qquad \forall j, k \in n, \forall i \in m \qquad if \qquad j\neq k$

In [34]:
for (j, k) in product(range(n), range(n)):
    if k != j:
        for i in range(m):
            model += X[j][i] - X[k][i] + M*Y[j][k][i] >= tempos_Proc[k][i]

$X_{i,k} \ge X_{i,j} + p_{i,j} - M(1 - Y_{i,j,k}), \qquad \forall j, k \in n, \forall i \in m \qquad if \qquad j\neq k$

In [35]:
for (j, k) in product(range(n), range(n)):
    if k != j:
        for i in range(m):
            model += -X[j][i] + X[k][i] - M*Y[j][k][i] >= tempos_Proc[j][i] - M

Chamando o solver

In [36]:
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

Impressão dos resultados

In [38]:

print("Completion time: ", C.x)
for (j, i) in product(range(n), range(m)):
    print("A tarefa %d inicia o seu processamento na máquina %d no tempo %g " % (j+1, i+1, X[j][i].x))

Completion time:  35.0
A tarefa 1 inicia o seu processamento na máquina 1 no tempo 9 
A tarefa 1 inicia o seu processamento na máquina 2 no tempo 0 
A tarefa 1 inicia o seu processamento na máquina 3 no tempo 14 
A tarefa 2 inicia o seu processamento na máquina 1 no tempo 0 
A tarefa 2 inicia o seu processamento na máquina 2 no tempo 22 
A tarefa 2 inicia o seu processamento na máquina 3 no tempo 28 
A tarefa 3 inicia o seu processamento na máquina 1 no tempo 24 
A tarefa 3 inicia o seu processamento na máquina 2 no tempo 14 
A tarefa 3 inicia o seu processamento na máquina 3 no tempo 0 
A tarefa 4 inicia o seu processamento na máquina 1 no tempo 22 
A tarefa 4 inicia o seu processamento na máquina 2 no tempo 7 
A tarefa 4 inicia o seu processamento na máquina 3 no tempo 24 
A tarefa 5 inicia o seu processamento na máquina 1 no tempo 14 
A tarefa 5 inicia o seu processamento na máquina 2 no tempo 27 
A tarefa 5 inicia o seu processamento na máquina 3 no tempo 2 


Impressão do Gannt

In [39]:
#Impressao do Gantt

df = []
colors =[]
for i in range(n):
    for j in range(m):
        s = str(datetime.datetime.strptime('2022-09-16 07:00:00', "%Y-%m-%d %H:%M:%S") + datetime.timedelta(minutes=np.float64(X[i][j].x) ))
        d = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")  + datetime.timedelta(minutes=np.float64(tempos_Proc[i][j]))
        df.append(dict(Task=str("maq "+str(j+1)), Start=str(s), Finish=str(d), Resource=str("job "+str(i+1))))
        r = lambda: random.randint(0,255)
        colors.append('#%02X%02X%02X' % (r(),r(),r()))



fig = ff.create_gantt(df, colors = colors, index_col='Resource', title = "JOB SHOP SCHEDULING - OPTIMUS SOLUTIONS LTDA ",show_colorbar=True, group_tasks=True,showgrid_x =True)    
fig.show()
